In [1]:
%load_ext autoreload
%autoreload 0

In [2]:
%autoreload

In [3]:
import sys
sys.path.append('/pollard/home/bsmith/Projects/haplo-benchmark/include/StrainFacts')

In [50]:
import xarray as xr
import sqlite3
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
import torch
import pyro
import scipy as sp

import lib.plot
from scipy.spatial.distance import pdist, squareform
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.cluster import AgglomerativeClustering
from lib.pandas_util import idxwhere
from tqdm import tqdm


import sfacts as sf

# from lib.project_style import color_palette, major_allele_frequency_bins
# from lib.project_data import metagenotype_db_to_xarray
# from lib.plot import ordination_plot, mds_ordination, nmds_ordination
# import lib.plot
# from lib.plot import construct_ordered_pallete
# from lib.pandas_util import idxwhere

In [5]:
threshold = 0.01  # We only care about droplets with >1% horizontal coverage
con = sqlite3.connect('data/ucfmt.db')
species_to_upper_limit_tally = pd.read_table('data/ucfmt.genotypes.distinct_positions_tally.tsv', index_col='species_id', squeeze=True)
species_to_reference_positions = pd.read_table('data/ucfmt.reference.distinct_positions_tally.tsv', index_col='species_id', squeeze=True)

In [6]:
species_upper_limit_horizontal_coverage = (species_to_upper_limit_tally / species_to_reference_positions).fillna(0)

In [7]:
species_to_consider = idxwhere(species_upper_limit_horizontal_coverage > threshold)

In [8]:
len(species_to_consider)

119

In [28]:
species_upper_limit_horizontal_coverage.sort_values()

species_id
100606    0.000000
102446    0.000000
102783    0.000000
102781    0.000000
102456    0.000000
            ...   
100268    0.818278
104345    0.990031
104018    1.000000
102763    1.000000
100878    1.000000
Length: 881, dtype: float64

In [45]:
focal_species = 100878
reference_length = species_to_reference_positions[focal_species]

In [46]:
focal_species_data = pd.read_sql(f"""
SELECT lib_id, species_position, reference_tally, alternative_tally, sample_id FROM snp_x_lib
JOIN lib USING (lib_id)
WHERE species_id = '{focal_species}' AND lib_type = 'droplet'
""", con=con)

In [47]:
barcode_tally_positions = focal_species_data.groupby('lib_id').apply(len)
barcode_tally_positions_horizontal_coverage = barcode_tally_positions / reference_length

In [48]:
passing_droplets = idxwhere(barcode_tally_positions_horizontal_coverage > threshold)
passing_droplets

['SS1057_TACCTCACTGTGAATGCC.d']

In [65]:
all_scg = {}

for focal_species in tqdm(idxwhere(species_upper_limit_horizontal_coverage > threshold)):
    focal_species_data = pd.read_sql(f"""
        SELECT lib_id, species_position, reference_tally, alternative_tally, sample_id
        FROM snp_x_lib
        JOIN lib USING (lib_id)
        WHERE species_id = '{focal_species}'
          AND lib_type = 'droplet'
    """, con=con)
    reference_length = species_to_reference_positions[focal_species]
    barcode_tally_positions = focal_species_data.groupby('lib_id').apply(len)
    barcode_tally_positions_horizontal_coverage = barcode_tally_positions / reference_length
    all_scg[focal_species] = barcode_tally_positions_horizontal_coverage
    
all_scg = pd.DataFrame(all_scg).stack(dropna=True)

100%|██████████| 119/119 [03:12<00:00,  1.62s/it]


In [77]:
all_scg

lib_id                             
SS1009_AACAACCTACAAGAGAGT.d  100199    0.000039
                             103695    0.001325
                             104296    0.000247
                             104345    0.000304
SS1009_AACAATGCAACACGTTGC.d  100113    0.000029
                                         ...   
SS1057_TTGTCAACCTGATACCGT.d  100199    0.000020
                             100268    0.000097
                             101157    0.000013
                             102274    0.000023
                             104016    0.000016
Length: 35066, dtype: float64

In [75]:
(
    all_scg
    .to_frame()
    .reset_index()
    .rename(columns={'level_1': 'species_id', 0: 'horizontal_coverage'})
    .sort_values(['species_id', 'horizontal_coverage'], ascending=(True, False))
    .to_csv('data/ucfmt.genotype.horizontal_coverage.tsv', sep='\t', index=False)
)

In [76]:
all_scg

lib_id                             
SS1009_AACAACCTACAAGAGAGT.d  100199    0.000039
                             103695    0.001325
                             104296    0.000247
                             104345    0.000304
SS1009_AACAATGCAACACGTTGC.d  100113    0.000029
                                         ...   
SS1057_TTGTCAACCTGATACCGT.d  100199    0.000020
                             100268    0.000097
                             101157    0.000013
                             102274    0.000023
                             104016    0.000016
Length: 35066, dtype: float64

In [82]:
scg_hc = pd.read_table('data/ucfmt.genotype.horizontal_coverage.tsv', index_col=['lib_id', 'species_id'], squeeze=True)

In [94]:
pd.read_table('data/ucfmt.genotype.horizontal_coverage.tsv', index_col=['lib_id', 'species_id'], squeeze=True)[lambda x: x > threshold].reset_index().species_id.value_counts()#.head(20)

104345    238
100199    147
100253     68
101387     62
102279     57
         ... 
101362      1
102072      1
104016      1
101711      1
103702      1
Name: species_id, Length: 87, dtype: int64

In [102]:
pd.read_table('data/ucfmt.genotype.horizontal_coverage.tsv', index_col=['lib_id', 'species_id'], squeeze=True)[lambda x: x > threshold].xs(100002, level='species_id')

lib_id
SS1057_TCATATCGCTCACTCGCT.d    0.034807
SS1009_CTGGACTAATAATGTAGC.d    0.024613
Name: horizontal_coverage, dtype: float64